# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This is a classification problem (a binary classification) since we need put(classify) all students into 2 categories: Pass or Fail. In case of regression, we would predict a continuos number(s) not classes. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [56]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print ("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [57]:
# TODO: Calculate number of students
n_students = student_data['passed'].count()

# TODO: Calculate number of features
n_features = student_data.columns[:-1].size

# TODO: Calculate passing students
n_passed = student_data[student_data['passed'] == 'yes']['passed'].count()

# TODO: Calculate failing students
n_failed = student_data[student_data['passed'] == 'no']['passed'].count()

# TODO: Calculate graduation rate
grad_rate = n_passed / float(n_students) * 100

# Print the results
print ("Total number of students: {}".format(n_students))
print ("Number of features: {}".format(n_features))
print ("Number of students who passed: {}".format(n_passed))
print ("Number of students who failed: {}".format(n_failed))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [58]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print ("Feature columns:\n{}".format(feature_cols))
print ("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print( "\nFeature values:")
print (X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [59]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print ("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [60]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_all = le.fit_transform(y_all)

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.24,
                                                    stratify=y_all, random_state=42)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))
print ("Grad rate of the training set: {:.2f}%".format(100 * (y_train == 1).mean()))
print ("Grad rate of the testing set: {:.2f}%".format(100 * (y_test == 1).mean()))

Training set has 300 samples.
Testing set has 95 samples.
Grad rate of the training set: 67.00%
Grad rate of the testing set: 67.37%


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: ** 


I have chosen the following supervised learning models which are appropriate fo this problem:
1. Logistic regression
2. RandomForestClassifier
3. Support Vector Machine(SVM)

All three models are widly used in different domains like recommendation system (netflix, amazon), marketing(to target a the most potential buyer), social media(to determing most influential people), personolized medicine/bio/chemoinformatics (to develop custom methods of treatment for a specific person), fraud detection(use by financial institutions and global payment systems), computer vision (image classification, quality control) and in many others.

*Strengths and Weaknesses*

Logistic regession (LR):

Unlike decision trees(RandomForestClassifier) or SVM, LR outpus well-calibrated predicted probabilities which is very convenient if you need to get ranking insted of biniray classification. Another strength side of LR is that the model can be easyly updated for taking in new data (online gradient descent method), again unlike RandomForestClassifier or SVM. Also, LG works well with small datasets, doesn't have a lot of parammeters to tune and the training time is fast.
On the other hand, LG presumes that features are in linear relationship and the log-odds of the response(although kernel can be used for non-liner relationships). LG cannot automatically learn feature interactions.

RandomForestClassifier(RF):

RF works well with both linear and non-linear problems. Also can be used for regression or classification. Can handle  high dimensional data and large number of training examples. Another advantage is that RF provides feature importance.
Here are some weaknesses: RF prones to overfitting (generalize data not so good compare to other algorithms).It also has more hyperparameters to tune.

SVM:

Works well with high dimentional feature space (text classification), can be use for linear and non-linear problems. Thr downside of SVM is that it is inefficient to train. So, it is not recommended to use SVM on a huge number of training examples.

All three model can be used with the provided data set because it has a small number of features, a small number of instances and the problem is linearly separable.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [61]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target, y_pred)#, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [62]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

# TODO: Initialize the three models
clf_A = LogisticRegression()
clf_B = RandomForestClassifier(random_state=42)
clf_C = LinearSVC(random_state=42)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

n_samples = [100, 200, 300]
clfs = [clf_A, clf_B, clf_C]


for n in n_samples:
    for clf in clfs:
        train_predict(clf, X_train[:n], y_train[: n], X_test, y_test)
        print("================================================================")
    print()
    

# TODO: Execute the 'train_predict' function for each classifier and each training set size


Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0016 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.8671.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7068.
Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0159 seconds
Made predictions in 0.0013 seconds.
F1 score for training set: 0.9924.
Made predictions in 0.0012 seconds.
F1 score for test set: 0.7368.
Training a LinearSVC using a training set size of 100. . .
Trained model in 0.0119 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8732.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.6880.

Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0023 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8211.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7391.
Training a RandomForestClassifier using a training s

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Logistic Regression**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0003             |         0.0002         |       0.8671     |      0.7068     |
| 200               |      0.0003             |         0.0003         |       0.8211     |      0.7391     |
| 300               |      0.0002             |         0.0002         |       0.8512     |      0.7500     |

** Classifer 2 - RandomForestClassifier**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0017             |         0.0028         |       0.9924     |      0.7368     |
| 200               |      0.0014             |         0.0012         |       0.9885     |      0.6822     |
| 300               |      0.0016             |         0.0013         |       0.9975     |      0.7132     |

** Classifer 3 - LinearSVC**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0004             |         0.0002         |       0.8732     |      0.6880     |
| 200               |      0.0003             |         0.0002         |       0.8227     |      0.7571     |
| 300               |      0.0005             |         0.0002         |       0.8256     |      0.6935     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

I thick that the best model, according to the data from the table, is Logistic Regression. It is fast to train and predict no matter how big the sample size is. It also has a good bias-variance trade-off meaning that it generalizes data well (better than RF or SVC). Linear SVC is also very fast but the F1 score is lower compare to Logistic Regression. On the other hand, RandomForestClassifier has lower F1 score and it tends to overfit data. In addition, it also 6-7 times slower than Logistic Regression (although for a small dataset it is not a big deal).

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **

Any machine learning(ML) model, in order to predict the future outcome requires data from the past. In our case, we have a list of students with certain caracteristics about each one. For examlpe, age, sex, studytime, health condition .etc. These  are called features. In addition to features, ML also needs to know weights for each feature. You can think of a weight as a degree of importance of a particular feature. For instance, studytime is more importand than sex of a student, that means the weight of studytime will be higher then the weight of sex feature. Do not worry about weights too much,  ML algorithm will automatically calculate them. Having features and weights, the model will use them to find relationship between them and the outcome (in other words, the model maps weighted features to binary answer). This process is called fitting the model(or training the model). After the model is trained, it intakes a list of new features which were not used in the training and predicts the outcome. Since we chose Logistic Regression as our best model, we will use it to predict the students performance.
Logistic Regression, unlike Linear Regression which assumes that data comes from a Gaussian distribution and predict numeric(continiuos) values, is a probalistic model based on Bernoulli distribution and predicts a probability of a particular event. Then, the probability is passed to a quantizer function that outputs a binary outcome(0 or 1). In Logistic regression the sigmoid function is used to calculate the probability by the following formula:
 $$F(z) = \frac{1} {1+e ^{-z}}$$
 
where $$ z = w_0 + w_1x_1 + w_2x_2 + + w_nn = w^Tx $$

w is a vector of weights, and x is a vector of features. The output(probabilities) of the sigmoid function is converted to a certain class using the following formula:

$$ \hat{y} = \begin{cases} 1 & \text{if  F(z)} ≥ 0.5 \\
0 &\text{otherwise.} \end{cases} $$

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [64]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.metrics import f1_score, make_scorer
from sklearn.grid_search import GridSearchCV


# TODO: Create the parameters list you wish to tune
parameters = {'C': (0.01, 0.1, 1, 10, 100),
             'penalty': ('l1', 'l2')}

# TODO: Initialize the classifier
clf = clf_A

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, verbose=1, scoring=f1_scorer)#scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
best_parameters = clf.get_params()

# Report the final F1 score for training and testing after parameter tuning
print ("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print ("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))
for param_name in sorted(parameters.keys()):
    print ('\t%s: %r' % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Made predictions in 0.0014 seconds.
Tuned model has a training F1 score of 0.8291.
Made predictions in 0.0004 seconds.
Tuned model has a testing F1 score of 0.7891.
	C: 0.1
	penalty: 'l1'


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.5s finished


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** 
The training score of the tuned model is 0.8291 and the testing score is 0.7891. Compare to the untuned model, the training score is a liitle higher( almost identical) and the testing score is 4% higher meaning that tuning the model improved result a little as well as generalization of the data.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.